<a href="https://colab.research.google.com/github/Sebory/Sklearn/blob/main/Exercise01/exercise01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!git clone https://github.com/Sebory/Sklearn.git

fatal: destination path 'Sklearn' already exists and is not an empty directory.


In [14]:
class Sentiment:
  NEGATIVE = "NEGATIVE"
  NEUTRAL = "NEUTRAL"
  POSITIVE = "POSITIVE"

class Review:
  def __init__(self, text, score):
    self.text = text
    self.score = score
    self.sentiment = self.get_sentiment()

  def get_sentiment(self):
    if self.score <= 2:
        return Sentiment.NEGATIVE
    elif self.score == 3:
        return Sentiment.NEUTRAL
    else: # score equal to 4 or 5
        return Sentiment.POSITIVE

# Load Data

In [15]:
import json

file_name = "/content/Sklearn/Exercise01/Books_small.json"

reviews = []
with open(file_name) as f:
  for line in f:
    review = json.loads(line)
    reviews.append(Review(review["reviewText"], review["overall"]))

reviews[23].score


4.0

# Prepare Data

In [16]:
len(reviews)

1000

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
training, test = train_test_split(reviews, test_size=0.33, random_state=42)

In [21]:
print(training[0].sentiment)

POSITIVE


In [24]:
print(test[0].sentiment)

POSITIVE


In [28]:
train_x = [x.text for x in training]
train_y = [x.sentiment for x in training]

test_x = [x.text for x in test]
test_y = [x.sentiment for x in test]

print(train_x[0])
print(train_y[0])

Vivid characters and descriptions. The author has created a tale that grabs your attention and I couldn't put it down.
POSITIVE


# Bag of words vectlorization

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

print(train_x[0])
print(train_x_vectors[0].toarray())

train_x_vectors
train_y

Vivid characters and descriptions. The author has created a tale that grabs your attention and I couldn't put it down.
[[0 0 0 ... 0 0 0]]
